# use session 
<b> session을 기본으로 하는 간단한 logistic regression을 만든다.  
<b> source 설명 이후 실습을 진행한다.  

In [1]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [2]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.keras import preprocessing

In [7]:
tokenizer = preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(samples) 
sequences = tokenizer.texts_to_sequences(samples)
sequences = preprocessing.sequence.pad_sequences(sequences, maxlen = 6, padding='post')

targets = np.array(targets)

word_index = tokenizer.word_index

print("index text data : \n", sequences)
print("shape of sequences:", sequences.shape)

index text data : 
 [[ 4  1  5  6  0  0]
 [ 7  1  8  9  0  0]
 [10  2  3 11  0  0]
 [12  2  3 13  0  0]
 [14  1 15 16  0  0]
 [17 18 19 20  0  0]]
shape of sequences: (6, 6)


In [18]:
inputs_ph = tf.placeholder(dtype=tf.int32, shape=[None, 6], name='sequences')
labels_ph = tf.placeholder(dtype=tf.int32, shape=[None, 1], name='labels')

In [27]:
epoch_size = 100
batch_size = 2

vocab_size = len(word_index) +1
emb_size = 128


# Embedding
embed_input = tf.keras.layers.Embedding(vocab_size, emb_size)(inputs_ph)
embed_input = tf.reduce_mean(embed_input, axis=-1)

# Model
hidden_layer = tf.keras.layers.Dense(128, activation=tf.nn.relu)(embed_input)
output_layer = tf.keras.layers.Dense(1)(hidden_layer)
output = tf.nn.sigmoid(output_layer)

# # Loss
loss = tf.losses.mean_squared_error(output, labels_ph)

# Optimizer
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [28]:
shuffle_sequence = []
shuffle_label = []
index = 0

sess = tf.Session()
#print("start time : ", start_time)
sess.run(tf.global_variables_initializer())


for epoch in range(epoch_size):
    random_index = np.random.permutation(len(sequences))
    shuffled_sequences = sequences[random_index]
    shuffled_targets = targets[random_index]
    for i in random_index[::batch_size]:
        _, _loss = sess.run([train_op, loss],
                             feed_dict={inputs_ph: shuffled_sequences[i:i+batch_size],
                                        labels_ph: shuffled_targets[i:i+batch_size]})
    if (epoch+1) % 10 == 0:
        print(f'# epoch{epoch} loss:{_loss}')   

    shuffle_sequence.clear()
    shuffle_label.clear()

# epoch0 loss:0.24945080280303955
# epoch1 loss:0.24633264541625977
# epoch2 loss:0.24837642908096313
# epoch3 loss:0.24770233035087585
# epoch4 loss:0.24650320410728455
# epoch5 loss:0.24806645512580872
# epoch6 loss:0.2298041731119156
# epoch7 loss:0.26520705223083496
# epoch8 loss:0.22161570191383362
# epoch9 loss:0.24207474291324615
# epoch10 loss:0.2143632173538208
# epoch11 loss:0.23774723708629608
# epoch12 loss:0.20544911921024323
# epoch13 loss:0.20222309231758118
# epoch14 loss:0.2335820198059082
# epoch15 loss:0.2712663412094116
# epoch16 loss:0.22631075978279114
# epoch17 loss:0.22274845838546753
# epoch18 loss:0.17809590697288513
# epoch19 loss:0.264899343252182
# epoch20 loss:0.2181352972984314
# epoch21 loss:0.21432460844516754
# epoch22 loss:0.2621909976005554
# epoch23 loss:0.19951876997947693
# epoch24 loss:0.14648136496543884
# epoch25 loss:0.19337934255599976
# epoch26 loss:0.18311923742294312
# epoch27 loss:0.18419481813907623
# epoch28 loss:0.16943511366844177
# e